In [61]:
import nltk
from nltk.corpus import twitter_samples

In [62]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/icaro/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [63]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [64]:
positives_tweets_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweets_tokens = twitter_samples.tokenized('negative_tweets.json')

In [65]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/icaro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/icaro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /home/icaro/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

In [67]:
def lemmatize_sentence(token):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(token):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word,pos))
    return lemmatized_sentence

In [68]:
print(positives_tweets_token[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [69]:
print(lemmatize_sentence(positives_tweets_token[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [70]:
import re , string

In [71]:
def remove_noise(tweets_tokens,stopwords = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweets_tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        tokens = lemmatizer.lemmatize(token,pos)
        if len(tokens)>0 and tokens not in string.punctuation and tokens.lower() not in stopwords:
            cleaned_tokens.append(tokens.lower())
    
    return cleaned_tokens

In [72]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/icaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
stop_words = stopwords.words('english')

In [74]:
print(remove_noise(positives_tweets_token[0],stopwords=stop_words))

['#followfriday', '@france_inte', '@pkuchly57', '@milipol_paris', 'top', 'engage', 'member', 'community', 'week', ':)']


In [75]:
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

In [76]:
for token in positives_tweets_token:
    positive_cleaned_tokens_list.append(remove_noise(token,stop_words))

In [77]:
for token in negative_tweets_tokens:
    negative_cleaned_tokens_list.append(remove_noise(token,stop_words))

In [78]:
print(positives_tweets_token[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']


In [79]:
print(positive_cleaned_tokens_list[500])

['dang', 'rad', '@abzugame', '#fanart', ':d', 'https://t.co/bi8k8tb9ht']


In [80]:
from nltk import FreqDist


In [81]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [82]:
all_pos_word = get_all_words(positive_cleaned_tokens_list)

In [83]:
freq_dist_pos =  FreqDist(all_pos_word)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [96]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens) 

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [97]:
import random 
positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model] 
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model] 
dataset = positive_dataset + negative_dataset 
random.shuffle(dataset) 
train_data = dataset[:7000]
test_data = dataset[7000:]

In [98]:

from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data) 
print("Accuracy is:", classify.accuracy(classifier, test_data)) 
print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2056.5 : 1.0
                      :) = True           Positi : Negati =    994.5 : 1.0
                     sad = True           Negati : Positi =     31.1 : 1.0
                     bam = True           Positi : Negati =     25.2 : 1.0
                follower = True           Positi : Negati =     22.4 : 1.0
               community = True           Positi : Negati =     19.2 : 1.0
                     x15 = True           Negati : Positi =     14.2 : 1.0
                  arrive = True           Positi : Negati =     13.7 : 1.0
                   enjoy = True           Positi : Negati =     13.4 : 1.0
               goodnight = True           Positi : Negati =     13.1 : 1.0
None
